# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""../../../../da…","""17272907547883…","""13124358176367…",1.2927e19,"""41056030405407…","""65875911555892…",false
"""../../../../da…","""11588847509576…","""11079288539490…",1.4856e19,"""41056030405407…","""10279081689292…",true
"""../../../../da…","""15788433388308…","""29854336548345…",5.6458e17,"""93830785523205…","""65875911555892…",false
"""../../../../da…","""13698405642463…","""11079288539490…",4.0723e18,"""77513098621506…","""65875911555892…",false
"""../../../../da…","""11383182230667…","""13124358176367…",1.4677e19,"""77513098621506…","""72851523257737…",true
"""../../../../da…","""46609049038663…","""29854336548345…",1.6330e19,"""16808307637162…",null,false
"""../../../../da…","""57210932764653…","""29854336548345…",8.7675e18,"""77513098621506…","""72851523257737…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_City': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""16534985523491…","""30830731771982…",1.2927e19,"""FirstMortgage3…","""10197197438983…","""Rejected"""
"""../../../../da…","""82382648602374…","""89381437376121…",1.4856e19,"""FirstMortgage3…","""13680217775912…","""Accepted"""
"""../../../../da…","""17098336089873…","""53660450324078…",5.6458e17,"""MoneyMarketSav…","""10197197438983…","""Rejected"""
"""../../../../da…","""35824746780978…","""89381437376121…",4.0723e18,"""BasicChecking""","""10197197438983…","""Rejected"""
"""../../../../da…","""12373839385439…","""30830731771982…",1.4677e19,"""BasicChecking""","""15309056341463…","""Accepted"""
"""../../../../da…","""10121462880196…","""53660450324078…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""14835093479837…","""53660450324078…",8.7675e18,"""BasicChecking""","""15309056341463…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""17802983640571…","""51395158818831…",1.2927e19,"""FirstMortgage3…","""89932684641520…","""Rejected"""
"""../../../../da…","""15866358061349…","""12193234551646…",1.4856e19,"""FirstMortgage3…","""12205613649409…","""Accepted"""
"""../../../../da…","""13525133427196…","""17643311666777…",5.6458e17,"""MoneyMarketSav…","""89932684641520…","""Rejected"""
"""../../../../da…","""11179373016584…","""12193234551646…",4.0723e18,"""BasicChecking""","""89932684641520…","""Rejected"""
"""../../../../da…","""43311010006896…","""51395158818831…",1.4677e19,"""BasicChecking""","""12701536632169…","""Accepted"""
"""../../../../da…","""69505775064715…","""17643311666777…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""29268760793648…","""17643311666777…",8.7675e18,"""BasicChecking""","""12701536632169…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_City=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""61958878299559…","""59408735566217…",1.2927e19,"""FirstMortgage3…","""15152104740935…","""Rejected"""
"""14495114431079…","""14593655763514…",1.4856e19,"""FirstMortgage3…","""15520884950214…","""Accepted"""
"""17510740810336…","""18424391332731…",5.6458e17,"""MoneyMarketSav…","""15152104740935…","""Rejected"""
"""14663382147339…","""14593655763514…",4.0723e18,"""BasicChecking""","""15152104740935…","""Rejected"""
"""60927148057217…","""59408735566217…",1.4677e19,"""BasicChecking""","""10419260060571…","""Accepted"""
"""54521596897132…","""18424391332731…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""64118049130810…","""18424391332731…",8.7675e18,"""BasicChecking""","""10419260060571…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'